In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

ton_iot = pd.read_csv('NF-ToN-IoT-v2.csv', nrows=1000)

ton_iot.shape

(1000, 45)

In [2]:
ton_iot.shape

(1000, 45)

### Deletes features from 

In [3]:
categorical_feature = ['L4_SRC_PORT','L4_DST_PORT','IPV4_SRC_ADDR',
                       'IPV4_DST_ADDR','PROTOCOL',
                       'L7_PROTO','DNS_QUERY_TYPE','ICMP_TYPE'
                        ,'ICMP_IPV4_TYPE','DNS_TTL_ANSWER']


not_used = ['DNS_QUERY_ID','FTP_COMMAND_RET_CODE']

tagets = ['Attack','Label']

to_drop_corr_col = ['SERVER_TCP_FLAGS', 'MAX_TTL', 'MAX_IP_PKT_LEN', 'RETRANSMITTED_OUT_PKTS']

to_drop_VIF = ['NUM_PKTS_1024_TO_1514_BYTES', 'CLIENT_TCP_FLAGS']

#to_drop = categorical_feature+not_used+tagets + to_drop_corr_col + to_drop_VIF

#X_Numerical = ton_iot.drop(columns=to_drop)

to_drop = not_used+tagets + to_drop_corr_col + to_drop_VIF

X = ton_iot.drop(columns=to_drop)


In [4]:
X['SRC_TO_DST_SECOND_BYTES'] = X['SRC_TO_DST_SECOND_BYTES'].astype(int)
X['DST_TO_SRC_SECOND_BYTES'] = X['DST_TO_SRC_SECOND_BYTES'].astype(int)

y_b = ton_iot['Label']


In [5]:
X.shape

(1000, 35)

## Scaling of Numerical value using  quantile

In [6]:
from sklearn.model_selection import train_test_split

X_train, X, y_train, y_b = train_test_split(X, y_b, test_size=0.1, random_state=42)



In [7]:
del ton_iot
del X_train
del y_train



In [8]:
X = X.reset_index()
y_b = y_b.reset_index()


In [9]:

X = X.drop(columns='index')
y_b = y_b.drop(columns='index')

In [10]:
X_Numerical = X.drop(columns=categorical_feature)

In [11]:
X_Numerical.head(4)

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,MIN_TTL,LONGEST_FLOW_PKT,...,RETRANSMITTED_IN_PKTS,RETRANSMITTED_OUT_BYTES,SRC_TO_DST_AVG_THROUGHPUT,DST_TO_SRC_AVG_THROUGHPUT,NUM_PKTS_UP_TO_128_BYTES,NUM_PKTS_128_TO_256_BYTES,NUM_PKTS_256_TO_512_BYTES,NUM_PKTS_512_TO_1024_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT
0,44,1,0,0,2,0,0,0,0,44,...,0,0,352000,0,1,0,0,0,1024,0
1,44,1,0,0,2,0,0,0,0,44,...,0,0,352000,0,1,0,0,0,1024,0
2,44,1,40,1,22,0,0,0,0,44,...,0,0,352000,320000,2,0,0,0,1024,0
3,44,1,0,0,2,0,0,0,0,44,...,0,0,352000,0,1,0,0,0,1024,0


In [12]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_Numerical_sc = pd.DataFrame()

scaler.fit(X_Numerical)

X_Numerical_sc[X_Numerical.columns] = scaler.transform(X_Numerical)

In [13]:
X_Numerical_sc.shape

(100, 25)

In [14]:
y_b.head(2)

,Label
0,0
1,0


In [15]:
print('Numerical')
print(25*0.8)
print(25*0.5)
print(25*0.2)
print('categorical')
print(10*0.8)
print(10*0.5)
print(10*0.2)

Numerical
20.0
12.5
5.0
categorical
8.0
5.0
2.0


## Scale data

In [16]:
from sklearn.feature_selection import f_classif, SelectKBest

sb = SelectKBest(f_classif, k=5)
sb.fit(X_Numerical_sc,y_b)
col_anova_20 = sb.get_feature_names_out()

##########################
sb = SelectKBest(f_classif, k=10)
sb.fit(X_Numerical_sc,y_b)
col_anova_40 = sb.get_feature_names_out()

##########################
sb = SelectKBest(f_classif, k=15)
sb.fit(X_Numerical_sc,y_b)
col_anova_60 = sb.get_feature_names_out()

In [17]:
sb.get_support()

array([False,  True,  True,  True,  True, False, False, False,  True,
        True, False, False, False,  True, False, False, False,  True,
        True,  True,  True,  True,  True,  True,  True])

In [18]:
X_A_20 = X_Numerical_sc[col_anova_20]
X_A_40 = X_Numerical_sc[col_anova_40]
X_A_60 = X_Numerical_sc[col_anova_60]

In [19]:
X_cat = X[categorical_feature]

# Optimize the features

### L4_SRC_PORT & L4_DST_PORT

Ports with numbers 0–1023 are called system or well-known ports; ports with numbers 1024-49151 are called user or registered ports, and ports with numbers 49152-65535 are called dynamic, private or ephemeral ports.[2] Both system and user ports are used by transport protocols (TCP, UDP, DCCP, SCTP) to identify an application or service.

In [20]:
well_known_port = list(range(1024))
registered_ports = list(range(1024,49152))
ephemeral_ports = list(range(49152,65535))

In [21]:
X_cat['L4_SRC_PORT_C'] = 'Others'
X_cat['L4_SRC_PORT_C'][X_cat['L4_SRC_PORT'].isin(well_known_port)] = 'well_known_port'
X_cat['L4_SRC_PORT_C'][X_cat['L4_SRC_PORT'].isin(registered_ports)] = 'registered_ports'
X_cat['L4_SRC_PORT_C'][X_cat['L4_SRC_PORT'].isin(ephemeral_ports)] = 'ephemeral_ports'

X_cat.drop(columns=['L4_SRC_PORT'],inplace=True)

X_cat['L4_SRC_PORT_C'].value_counts()

print(X_cat.shape)

(100, 10)


In [22]:
X_cat['L4_DST_PORT_C'] = 'Others'
X_cat['L4_DST_PORT_C'][X_cat['L4_DST_PORT'].isin(well_known_port)] = 'well_known_port'
X_cat['L4_DST_PORT_C'][X_cat['L4_DST_PORT'].isin(registered_ports)] = 'registered_ports'
X_cat['L4_DST_PORT_C'][X_cat['L4_DST_PORT'].isin(ephemeral_ports)] = 'ephemeral_ports'

X_cat.drop(columns=['L4_DST_PORT'],inplace=True)
X_cat['L4_DST_PORT_C'].value_counts()


registered_ports    80
well_known_port     16
ephemeral_ports      4
Name: L4_DST_PORT_C, dtype: int64

#### Removerd features are :

to_drop_zero = ['RETRANSMITTED_IN_BYTES','RETRANSMITTED_IN_PKTS','RETRANSMITTED_OUT_BYTES','FTP_COMMAND_RET_CODE','ICMP_TYPE','DNS_TTL_ANSWER']
&&

['SERVER_TCP_FLAGS', 'MAX_TTL', 'MAX_IP_PKT_LEN', 'RETRANSMITTED_OUT_PKTS', 'ICMP_IPV4_TYPE']

## DNS

In [23]:
X_cat['DNS_QUERY_TYPE'] = X_cat['DNS_QUERY_TYPE'].astype('O')
list_proto = [0,1,2,12]
X_cat['DNS_QUERY_TYPE'][~X_cat['DNS_QUERY_TYPE'].isin(list_proto)] = 'Other'
X_cat['DNS_QUERY_TYPE'].value_counts()

0    98
1     2
Name: DNS_QUERY_TYPE, dtype: int64

### PROTOCOL

In [24]:
X_cat['PROTOCOL'] = X_cat['PROTOCOL'].astype(int)

X_cat['PROTOCOL'].value_counts()

6     89
17    11
Name: PROTOCOL, dtype: int64

In [25]:
print(X_cat.shape)

(100, 10)


In [26]:
X_cat['PROTOCOL'].value_counts()

6     89
17    11
Name: PROTOCOL, dtype: int64

### L7_PROTO

In [27]:
X_cat['L7_PROTO'] = X_cat['L7_PROTO'].astype('O')
list_proto = [0,7,91,7.178,1,92,131,5.178]
X_cat['L7_PROTO'][~X_cat['L7_PROTO'].isin(list_proto)] = 'Other'
X_cat['L7_PROTO'].value_counts()

0.0      93
Other     4
92.0      3
Name: L7_PROTO, dtype: int64

#### Adresses

### Adress Range

In [28]:
# Private adresses
# Class A: 10.0.0.0 to 10.255.255.255
# Class B: 172.16.0.0 to 172.31.255.255
# Class C: 192.168.0.0 to 192.168.255.255
from ipaddress import ip_address


def findIPs(start, end):
    start = ip_address(start)
    end = ip_address(end)
    result = []
    while start <= end:
        result.append(str(start))
        start += 1
    return result
print('classe_a process ------------------------------->')
plage_classe_a=findIPs('10.0.0.0', '10.255.255.255')
print('classe_b process ------------------------------->')
plage_classe_b=findIPs('172.16.0.0', '172.31.255.255')
print('classe_c process ------------------------------->')
plage_classe_c=findIPs('192.168.0.0', '192.168.255.255')

classe_a process ------------------------------->
classe_b process ------------------------------->
classe_c process ------------------------------->


In [29]:
import numpy as np
X_cat['IPV4_DST_ADDR_CLASSE'] = 'None'
print('classe_c process ------------------------------->')
X_cat['IPV4_DST_ADDR_CLASSE'][X_cat['IPV4_DST_ADDR'].isin(plage_classe_c)] = 'classe_c'
print('classe_b process ------------------------------->')
X_cat['IPV4_DST_ADDR_CLASSE'][X_cat['IPV4_DST_ADDR'].isin(plage_classe_b)] = 'classe_b'
print('classe_a process ------------------------------->')
X_cat['IPV4_DST_ADDR_CLASSE'][X_cat['IPV4_DST_ADDR'].isin(plage_classe_a)] = 'classe_a'
print('localhost process ------------------------------->')
X_cat['IPV4_DST_ADDR_CLASSE'][X_cat['IPV4_DST_ADDR'] == '127.0.0.1'] = 'localhost'
print('public/other process ------------------------------->')
X_cat['IPV4_DST_ADDR_CLASSE'][X_cat['IPV4_DST_ADDR_CLASSE'] == 'None'] = 'public/other'
X_cat['IPV4_DST_ADDR_CLASSE'].value_counts()

classe_c process ------------------------------->
classe_b process ------------------------------->
classe_a process ------------------------------->
localhost process ------------------------------->
public/other process ------------------------------->


classe_c        94
public/other     6
Name: IPV4_DST_ADDR_CLASSE, dtype: int64

### IPV4_SRC_ADDR_CLASSE

In [30]:
import numpy as np
X_cat['IPV4_SRC_ADDR_CLASSE'] = 'None'
print('classe_c process ------------------------------->')
X_cat['IPV4_SRC_ADDR_CLASSE'][X_cat['IPV4_SRC_ADDR'].isin(plage_classe_c)] = 'classe_c'
print('classe_b process ------------------------------->')
X_cat['IPV4_SRC_ADDR_CLASSE'][X_cat['IPV4_SRC_ADDR'].isin(plage_classe_b)] = 'classe_b'
print('classe_a process ------------------------------->')
X_cat['IPV4_SRC_ADDR_CLASSE'][X_cat['IPV4_SRC_ADDR'].isin(plage_classe_a)] = 'classe_a'
print('localhost process ------------------------------->')
X_cat['IPV4_SRC_ADDR_CLASSE'][X_cat['IPV4_SRC_ADDR'] == '127.0.0.1'] = 'localhost'
print('public/other process ------------------------------->')
X_cat['IPV4_SRC_ADDR_CLASSE'][X_cat['IPV4_SRC_ADDR_CLASSE'] == 'None'] = 'public/other'
X_cat['IPV4_SRC_ADDR_CLASSE'].value_counts()

classe_c process ------------------------------->
classe_b process ------------------------------->
classe_a process ------------------------------->
localhost process ------------------------------->
public/other process ------------------------------->


classe_c    99
classe_b     1
Name: IPV4_SRC_ADDR_CLASSE, dtype: int64

Remove IPV4_SRC_ADDR & IPV4_DST_ADDR and Keep IPV4_SRC_ADDR IPV4_DST_ADDR

In [31]:
X_cat.drop(columns=['IPV4_SRC_ADDR','IPV4_DST_ADDR'], inplace=True)
X.drop(columns=['IPV4_SRC_ADDR','IPV4_DST_ADDR'], inplace=True)

In [32]:
X_cat.head(2)

,PROTOCOL,L7_PROTO,DNS_QUERY_TYPE,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_TTL_ANSWER,L4_SRC_PORT_C,L4_DST_PORT_C,IPV4_DST_ADDR_CLASSE,IPV4_SRC_ADDR_CLASSE
0,6,0.0,0,0,0,0,registered_ports,registered_ports,classe_c,classe_c
1,6,0.0,0,0,0,0,ephemeral_ports,registered_ports,classe_c,classe_c


## Optimze the 3 features ICMP_TYPE, ICMP_IPV4_TYPE, DNS_TTL_ANSWER

In [33]:
import math
infini = math.inf

X_cat['ICMP_TYPE'] = X_cat['ICMP_TYPE'].astype('O')
X_cat['ICMP_IPV4_TYPE'] = X_cat['ICMP_IPV4_TYPE'].astype('O')
X_cat['DNS_TTL_ANSWER'] = X_cat['DNS_TTL_ANSWER'].astype('O')

X_cat['ICMP_TYPE'][~(X_cat['ICMP_TYPE']==0)]=1
X_cat['ICMP_IPV4_TYPE'][~(X_cat['ICMP_IPV4_TYPE']==0)]=1
X_cat['DNS_TTL_ANSWER'][~(X_cat['DNS_TTL_ANSWER']==0)]=1

print(f"ICMPTYPE {X_cat['ICMP_TYPE'].value_counts()}, \n ICMP_IPV4_TYPE  {X_cat['ICMP_IPV4_TYPE'].value_counts()}, \n  DNS_TTL_ANSWER  {X_cat['DNS_TTL_ANSWER'].value_counts()}")



ICMPTYPE 0    100
Name: ICMP_TYPE, dtype: int64, 
 ICMP_IPV4_TYPE  0    100
Name: ICMP_IPV4_TYPE, dtype: int64, 
  DNS_TTL_ANSWER  0    98
1     2
Name: DNS_TTL_ANSWER, dtype: int64


In [34]:
X_cat.head(4)

,PROTOCOL,L7_PROTO,DNS_QUERY_TYPE,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_TTL_ANSWER,L4_SRC_PORT_C,L4_DST_PORT_C,IPV4_DST_ADDR_CLASSE,IPV4_SRC_ADDR_CLASSE
0,6,0.0,0,0,0,0,registered_ports,registered_ports,classe_c,classe_c
1,6,0.0,0,0,0,0,ephemeral_ports,registered_ports,classe_c,classe_c
2,6,0.0,0,0,0,0,ephemeral_ports,registered_ports,classe_c,classe_c
3,6,0.0,0,0,0,0,ephemeral_ports,registered_ports,classe_c,classe_c


# Feature Selection Filter-based

In [35]:
from sklearn.feature_selection import chi2

# Convert string values in categorical features to numerical
X_cat = X_cat.replace('Other',999)
X_cat = X_cat.replace('Others',999)
X_cat = X_cat.replace('well_known_port',0)
X_cat = X_cat.replace('registered_ports',1)
X_cat = X_cat.replace('ephemeral_ports',2)



X_cat = X_cat.replace('localhost',0)
X_cat = X_cat.replace('classe_a',1)
X_cat = X_cat.replace('classe_b',2)
X_cat = X_cat.replace('classe_c',3)
X_cat = X_cat.replace('public/other',4)


X_cat = X_cat.astype(int)



#X.dropna(inplace=True)

#X = X.astype(int)




In [36]:
X_cat_int = X_cat.astype(int)

In [37]:
X_cat_int.head(2)

,PROTOCOL,L7_PROTO,DNS_QUERY_TYPE,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_TTL_ANSWER,L4_SRC_PORT_C,L4_DST_PORT_C,IPV4_DST_ADDR_CLASSE,IPV4_SRC_ADDR_CLASSE
0,6,0,0,0,0,0,1,1,3,3
1,6,0,0,0,0,0,2,1,3,3


In [38]:
chi2_corr = chi2(X_cat,y_b)

In [39]:
list(X_cat.columns[np.argsort(-chi2_corr[0])])

['L7_PROTO',
 'PROTOCOL',
 'L4_DST_PORT_C',
 'L4_SRC_PORT_C',
 'DNS_QUERY_TYPE',
 'DNS_TTL_ANSWER',
 'IPV4_DST_ADDR_CLASSE',
 'IPV4_SRC_ADDR_CLASSE',
 'ICMP_TYPE',
 'ICMP_IPV4_TYPE']

In [40]:
columns_ranked_chi2 = list(X_cat.columns[np.argsort(-chi2_corr[0])])

## Ranking of features using CHI2

# We will select 3 pourcentage from the rest of the features
categorical
- 6.0
- 4.0
- 2.0

### Generate the 3 databases for chi 2

In [41]:
X_chi2_20 = X_cat[columns_ranked_chi2[:2]]
X_chi2_40 =  X_cat[columns_ranked_chi2[:4]]
X_chi2_60 =  X_cat[columns_ranked_chi2[:6]]

dummy_feature = ['L4_SRC_PORT_C','L4_DST_PORT_C','IPV4_SRC_ADDR_CLASSE',
                 'IPV4_DST_ADDR_CLASSE','PROTOCOL','L7_PROTO']
list_dummy_20 = list(set( list(X_chi2_20.columns)).intersection(dummy_feature))
list_dummy_40 = list(set( list(X_chi2_40.columns)).intersection(dummy_feature))
list_dummy_60 = list(set( list(X_chi2_60.columns)).intersection(dummy_feature))


X_chi2_20 = pd.get_dummies(X_chi2_20, columns=list_dummy_20)
X_chi2_40 = pd.get_dummies(X_chi2_40, columns=list_dummy_40)
X_chi2_60 = pd.get_dummies(X_chi2_60, columns=list_dummy_60)

In [42]:
dummy_feature = ['L4_SRC_PORT_C','L4_DST_PORT_C','IPV4_SRC_ADDR_CLASSE',
                 'IPV4_DST_ADDR_CLASSE','PROTOCOL','L7_PROTO']
X_cat_dm = pd.get_dummies(X_cat, columns=dummy_feature)
X =  pd.concat([X_Numerical_sc,X_cat_dm],axis=1)
X.shape

(100, 44)

In [58]:
X_chi2_20

,PROTOCOL_6,PROTOCOL_17,L7_PROTO_0,L7_PROTO_92,L7_PROTO_999
0,1,0,1,0,0
1,1,0,1,0,0
2,1,0,1,0,0
3,1,0,1,0,0
4,1,0,1,0,0
...,...,...,...,...,...
95,1,0,1,0,0
96,1,0,1,0,0
97,1,0,1,0,0
98,0,1,1,0,0


In [59]:
X_A_40

,OUT_BYTES,OUT_PKTS,LONGEST_FLOW_PKT,DST_TO_SRC_SECOND_BYTES,SRC_TO_DST_AVG_THROUGHPUT,DST_TO_SRC_AVG_THROUGHPUT,NUM_PKTS_UP_TO_128_BYTES,NUM_PKTS_128_TO_256_BYTES,NUM_PKTS_512_TO_1024_BYTES,TCP_WIN_MAX_IN
0,-0.170883,-0.222438,-0.285868,-0.170883,-0.246378,-0.170883,-0.284364,-0.171815,-0.190219,-0.186624
1,-0.170883,-0.222438,-0.285868,-0.170883,-0.246378,-0.170883,-0.284364,-0.171815,-0.190219,-0.186624
2,-0.167036,-0.181016,-0.285868,-0.167036,-0.246378,-0.167036,-0.191434,-0.171815,-0.190219,-0.186624
3,-0.170883,-0.222438,-0.285868,-0.170883,-0.246378,-0.170883,-0.284364,-0.171815,-0.190219,-0.186624
4,-0.170883,-0.222438,-0.285868,-0.170883,-0.246378,-0.170883,-0.284364,-0.171815,-0.190219,-0.186624
...,...,...,...,...,...,...,...,...,...,...
95,-0.170883,-0.222438,-0.285868,-0.170883,-0.246378,-0.170883,-0.284364,-0.171815,-0.190219,-0.186624
96,-0.167036,-0.181016,-0.285868,-0.167036,-0.246378,-0.167036,-0.191434,-0.171815,-0.190219,-0.186624
97,-0.036230,1.227346,3.144107,-0.036230,0.724435,-0.036230,3.525736,-0.015145,3.791108,-0.361194
98,-0.170883,-0.222438,-0.233607,-0.170883,-0.245811,-0.170883,-0.284364,-0.171815,-0.190219,-0.413765


In [60]:
X.columns

NameError: name 'X' is not defined

## mRMR

In [46]:
from mrmr import mrmr_classif
selected_features_20 = mrmr_classif(X=X_cat, y=y_b, K=2)
selected_features_40 = mrmr_classif(X=X_cat, y=y_b, K=4)
selected_features_60 = mrmr_classif(X=X_cat, y=y_b, K=6)

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:04<00:00,  1.22it/s]


In [56]:
X_mrmr_60 = X_cat[selected_features_60]
X_mrmr_40 = X_cat[selected_features_40]
X_mrmr_20 = X_cat[selected_features_20]

list_dummy_20 = list(set( list(X_mrmr_20.columns)).intersection(dummy_feature))
list_dummy_40 = list(set( list(X_mrmr_40.columns)).intersection(dummy_feature))
list_dummy_60 = list(set( list(X_mrmr_60.columns)).intersection(dummy_feature))

X_mrmr_20 = pd.get_dummies(X_mrmr_20, columns=list_dummy_20)
X_mrmr_40 = pd.get_dummies(X_mrmr_40, columns=list_dummy_40)
X_mrmr_60 = pd.get_dummies(X_mrmr_60, columns=list_dummy_60)

In [57]:
X.head(4)

NameError: name 'X' is not defined

### Decision Tree

In [61]:
X_chi2_60 = X_chi2_60.astype(int)
X_chi2_40 = X_chi2_40.astype(int)
X_chi2_20 = X_chi2_20.astype(int)



X_mrmr_60 = X_mrmr_60.astype(int)
X_mrmr_40 = X_mrmr_40.astype(int)
X_mrmr_20 = X_mrmr_20.astype(int)

X_cat = X_cat.astype(int)

In [62]:
X_cat

,PROTOCOL,L7_PROTO,DNS_QUERY_TYPE,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_TTL_ANSWER,L4_SRC_PORT_C,L4_DST_PORT_C,IPV4_DST_ADDR_CLASSE,IPV4_SRC_ADDR_CLASSE
0,6,0,0,0,0,0,1,1,3,3
1,6,0,0,0,0,0,2,1,3,3
2,6,0,0,0,0,0,2,1,3,3
3,6,0,0,0,0,0,2,1,3,3
4,6,0,0,0,0,0,1,1,3,3
...,...,...,...,...,...,...,...,...,...,...
95,6,0,0,0,0,0,1,1,3,3
96,6,0,0,0,0,0,1,1,3,3
97,6,0,0,0,0,0,1,2,3,3
98,17,0,0,0,0,0,2,1,4,3


In [63]:
X_A_chi2_20 = pd.concat([X_A_20,X_chi2_20],axis=1)
X_A_chi2_40 = pd.concat([X_A_40,X_chi2_40],axis=1)
X_A_chi2_60 = pd.concat([X_A_60,X_chi2_60],axis=1)


X_A_mRMR_20 = pd.concat([X_A_20,X_mrmr_20],axis=1)
X_A_mRMR_40 = pd.concat([X_A_40,X_mrmr_40],axis=1)
X_A_mRMR_60 = pd.concat([X_A_60,X_mrmr_60],axis=1)


# Insert the new code for import here

In [65]:
datasets = {"X_A_chi2_20":X_A_chi2_20,
"X_A_chi2_40":X_A_chi2_40,
"X_A_chi2_60":X_A_chi2_60,

"X_A_mRMR_20":X_A_mRMR_20,
"X_A_mRMR_40":X_A_mRMR_40,
"X_A_mRMR_60":X_A_mRMR_60,
            }

In [66]:
del X_A_chi2_20,
del X_A_chi2_40,
del X_A_chi2_60,

del X_A_mRMR_20,
del X_A_mRMR_40,
del X_A_mRMR_60
            


NameError: name 'X' is not defined

In [96]:
X[-1] = np.zeros(len(X[-1]))

In [97]:
X

array([[ 0.,  0.,  5., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ..., 10.,  0.,  0.],
       [ 0.,  0.,  1., ...,  3.,  0.,  0.],
       ...,
       [ 0.,  0.,  5., ...,  8.,  1.,  0.],
       [ 0.,  0.,  6., ...,  4.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [92]:
import xgboost

from xgboost import  XGBClassifier

from micromlgen import port
import micromlgen


print(xgboost.__version__)


clf = XGBClassifier(random_state=0, n_estimators = 5)
clf.fit(datasets['X_A_chi2_20'],one_hot_encoded_tmp)
contenent = port(clf, tmp_file='./model.json')


1.7.1


ValueError: Invalid classes inferred from unique values of `y`.  Expected: [0 1], got [1. 2.]

In [56]:
datasets['X_A_chi2_20'].columns

Index(['OUT_PKTS', 'LONGEST_FLOW_PKT', 'SRC_TO_DST_AVG_THROUGHPUT',
       'NUM_PKTS_UP_TO_128_BYTES', 'TCP_WIN_MAX_IN', 'L7_PROTO_0',
       'L7_PROTO_92', 'L7_PROTO_999', 'PROTOCOL_6', 'PROTOCOL_17'],
      dtype='object')

In [58]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier()

clf.fit(datasets['X_A_chi2_20'],y_b)

DecisionTreeClassifier()

In [59]:
contenent = port(clf)


In [107]:
from micromlgen import port

def create_folder(path):
    path_temp = Path(path)
    path_temp.mkdir(parents=True)
    
import sys
def save_my_model(path,name_file,extension,contenent):
    orig_stdout = sys.stdout
    f = open(path+'/'+name_file+'.'+extension , 'w')
    sys.stdout = f
    print(contenent)
    sys.stdout = orig_stdout
    f.close()

In [110]:
from sklearn.ensemble import RandomForestClassifier
import os 
from pathlib import Path


file_names = []
for k, data in datasets.items():
    for  n_estimators in [5, 20, 40, 70, 100] :
        clf = RandomForestClassifier(random_state=0, n_estimators = n_estimators)
        clf.fit(data,y_b)
        contenent = port(clf)
        k = k.replace("X_", "")
        file_name= k + "_RF"+"_"+str(n_estimators)
        file_names.append(file_name)
        if not os.path.exists('tinyml_result_ton/RF/'+file_name):
            create_folder('tinyml_result_ton/RF/'+file_name)
        save_my_model("tinyml_result_ton/RF/"+file_name,file_name,'h',contenent)
        

In [111]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import  XGBClassifier

import os 
from pathlib import Path

y = y_b.copy()
y = y.to_numpy()
y[-1] = y.max() + 1

file_names = []
for k, data in datasets.items():
    for  n_estimators in [5, 20, 40, 70, 100] :
        clf = XGBClassifier(random_state=0, n_estimators = n_estimators)
        clf.fit(data,y)
        contenent = port(clf, tmp_file='./model.json')
        k = k.replace("X_", "")
        file_name= k + "_XGB"+"_"+str(n_estimators)
        file_names.append(file_name)
        if not os.path.exists('tinyml_result_ton/XGB/'+file_name):
            create_folder('tinyml_result_ton/XGB/'+file_name)
        save_my_model("tinyml_result_ton/XGB/"+file_name,file_name,'h',contenent)
